In [1]:
# pip install transformers datasets torch pandas transformers[torch]

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import time

/Users/enriccogemha/Downloads/hpc/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("sst2") # Example: Stanford Sentiment Treebank-2 dataset
train_dataset = dataset["train"].shuffle(seed=42).select(range(1000)) # Subset for faster local training

In [4]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

/Users/enriccogemha/Downloads/hpc/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = train_dataset.map(tokenize_function, batched=True)

In [6]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

In [7]:
start_time = time.time()
trainer.train()
end_time = time.time()

100%|██████████| 189/189 [01:52<00:00,  1.68it/s]

{'train_runtime': 112.3997, 'train_samples_per_second': 26.69, 'train_steps_per_second': 1.681, 'train_loss': 0.26441761804005454, 'epoch': 3.0}


In [8]:
print(f"Local Training Time: {end_time - start_time} seconds")

Local Training Time: 112.59222388267517 seconds
